<a href="https://colab.research.google.com/github/shhesterka04/q_prj/blob/main/Deutsch_Josza.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Реализация алгоритма Дойча-Йожи

In [ ]:
!pip install qiskit --quiet

In [4]:
import numpy as np
from qiskit import QuantumCircuit, Aer, transpile, assemble
from qiskit.visualization import plot_histogram
from qiskit.circuit.library import Permutation

def oracle(constant, input_size, permute=None):
    circuit = QuantumCircuit(input_size + 1)

    if not constant:
        if permute is None:
            permute = np.random.permutation(2 ** input_size)

        permutation_gate = Permutation(input_size, permute).control(1)
        circuit.append(permutation_gate, range(input_size + 1))

    return circuit

def deutsch_josza(input_size, oracle):
    circuit = QuantumCircuit(input_size + 1, input_size)
    
    circuit.x(input_size)
    circuit.barrier()
    
    for i in range(input_size + 1):
        circuit.h(i)

    circuit.barrier()
    circuit.compose(oracle, inplace=True)
    
    circuit.barrier()
    for i in range(input_size):
        circuit.h(i)

    circuit.barrier()
    for i in range(input_size):
        circuit.measure(i, i)

    return circuit

def test_deutsch_josza(input_size, constant, permute=None):
    test_oracle = oracle(constant, input_size, permute)
    test_circuit = deutsch_josza(input_size, test_oracle)
    qasm_backend = Aer.get_backend('qasm_simulator')
    tqc = transpile(test_circuit, qasm_backend)
    qobj = assemble(tqc, shots=1024)
    results = qasm_backend.run(qobj).result()
    counts = results.get_counts()

    if '0' * input_size in counts:
        print("The function is constant.")
    else:
        print("The function is balanced.")

if __name__ == '__main__':
    input_size = 3

    # Test with a constant function
    test_deutsch_josza(input_size, constant=True)

    # Test with a balanced function
    test_deutsch_josza(input_size, constant=False)


<ipython-input-4-8e93f4a3eba3>:46: DeprecationWarning: Using a qobj for run() is deprecated as of qiskit-aer 0.9.0 and will be removed no sooner than 3 months from that release date. Transpiled circuits should now be passed directly using `backend.run(circuits, **run_options).
  results = qasm_backend.run(qobj).result()


TypeError: ignored